## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmaps_api_key

# Configure gmaps API key
gmaps.configure(api_key=gmaps_api_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,74.68,60,1,5.75,clear sky
1,1,Ostrovnoy,RU,68.0531,39.5131,46.36,96,97,4.72,overcast clouds
2,2,Provideniya,RU,64.3833,-173.3000,46.02,80,95,4.63,overcast clouds
3,3,Kahului,US,20.8947,-156.4700,86.02,63,20,29.93,few clouds
4,4,Banjar,ID,-8.1900,114.9675,78.66,86,44,5.59,scattered clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your trip? "))
max_temp = float(input("What is the maximum temperature for your trip? "))

What is the minimum temperature for your trip? 50
What is the maximum temperature for your trip? 80


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
pref_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,74.68,60,1,5.75,clear sky
4,4,Banjar,ID,-8.1900,114.9675,78.66,86,44,5.59,scattered clouds
6,6,Ferrol,ES,43.4832,-8.2369,62.67,80,30,6.76,scattered clouds
7,7,Praya,CV,14.9215,-23.5087,77.54,78,20,12.66,few clouds
8,8,Saint-Philippe,RE,-21.3585,55.7679,65.75,84,36,10.96,light rain
9,9,Bredasdorp,ZA,-34.5322,20.0403,53.28,89,100,4.61,overcast clouds
10,10,Airai,TL,-8.9266,125.4092,61.25,74,57,2.93,broken clouds
11,11,Kawalu,ID,-7.3817,108.2082,71.26,97,89,1.19,overcast clouds
12,12,Port Alfred,ZA,-33.5906,26.8910,65.86,34,100,7.94,overcast clouds
13,13,Hermanus,ZA,-34.4187,19.2345,53.91,88,100,6.02,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
pref_cities_df.count()

City_ID                445
City                   445
Country                441
Lat                    445
Lng                    445
Max Temp               445
Humidity               445
Cloudiness             445
Wind Speed             445
Current Description    445
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df.dropna(subset = ["Country"], inplace = True)
pref_cities_df.count()

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


City_ID                441
City                   441
Country                441
Lat                    441
Lng                    441
Max Temp               441
Humidity               441
Cloudiness             441
Wind Speed             441
Current Description    441
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,74.68,clear sky,41.3712,-73.4140,
4,Banjar,ID,78.66,scattered clouds,-8.1900,114.9675,
6,Ferrol,ES,62.67,scattered clouds,43.4832,-8.2369,
7,Praya,CV,77.54,few clouds,14.9215,-23.5087,
8,Saint-Philippe,RE,65.75,light rain,-21.3585,55.7679,
9,Bredasdorp,ZA,53.28,overcast clouds,-34.5322,20.0403,
10,Airai,TL,61.25,broken clouds,-8.9266,125.4092,
11,Kawalu,ID,71.26,overcast clouds,-7.3817,108.2082,
12,Port Alfred,ZA,65.86,overcast clouds,-33.5906,26.8910,
13,Hermanus,ZA,53.91,overcast clouds,-34.4187,19.2345,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_api_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    city = row["City"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    gmaps_api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Getting hotel for locations provided to city {city}")
    hotels = requests.get(gmaps_api_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found !!")
    except (IndexError):
        print("Hotel not found ... skipping.")
        

Getting hotel for locations provided to city Bethel
Hotel found !!
Getting hotel for locations provided to city Banjar
Hotel found !!
Getting hotel for locations provided to city Ferrol
Hotel found !!
Getting hotel for locations provided to city Praya
Hotel found !!
Getting hotel for locations provided to city Saint-Philippe
Hotel found !!
Getting hotel for locations provided to city Bredasdorp
Hotel found !!
Getting hotel for locations provided to city Airai
Hotel found !!
Getting hotel for locations provided to city Kawalu
Hotel found !!
Getting hotel for locations provided to city Port Alfred
Hotel found !!
Getting hotel for locations provided to city Hermanus
Hotel found !!
Getting hotel for locations provided to city Alyangula
Hotel found !!
Getting hotel for locations provided to city Ostersund
Hotel found !!
Getting hotel for locations provided to city Bengkulu
Hotel found !!
Getting hotel for locations provided to city Atuona
Hotel found !!
Getting hotel for locations provided 

Hotel not found ... skipping.
Getting hotel for locations provided to city Gweta
Hotel found !!
Getting hotel for locations provided to city Kotka
Hotel found !!
Getting hotel for locations provided to city Hualmay
Hotel found !!
Getting hotel for locations provided to city Bagdarin
Hotel found !!
Getting hotel for locations provided to city Constitucion
Hotel found !!
Getting hotel for locations provided to city Port Elizabeth
Hotel found !!
Getting hotel for locations provided to city Ola
Hotel not found ... skipping.
Getting hotel for locations provided to city Pilas
Hotel found !!
Getting hotel for locations provided to city Darhan
Hotel found !!
Getting hotel for locations provided to city Richards Bay
Hotel found !!
Getting hotel for locations provided to city Kruisfontein
Hotel found !!
Getting hotel for locations provided to city Qasigiannguit
Hotel found !!
Getting hotel for locations provided to city Taybad
Hotel found !!
Getting hotel for locations provided to city Talnakh
H

Hotel found !!
Getting hotel for locations provided to city Lugovoy
Hotel found !!
Getting hotel for locations provided to city Nanga Eboko
Hotel found !!
Getting hotel for locations provided to city Boddam
Hotel found !!
Getting hotel for locations provided to city Khandyga
Hotel found !!
Getting hotel for locations provided to city Mamburao
Hotel found !!
Getting hotel for locations provided to city Artigas
Hotel found !!
Getting hotel for locations provided to city Mozarlandia
Hotel found !!
Getting hotel for locations provided to city Yumen
Hotel not found ... skipping.
Getting hotel for locations provided to city Krasnoufimsk
Hotel found !!
Getting hotel for locations provided to city Saldanha
Hotel found !!
Getting hotel for locations provided to city Great Yarmouth
Hotel found !!
Getting hotel for locations provided to city Luwuk
Hotel found !!
Getting hotel for locations provided to city Sai Buri
Hotel found !!
Getting hotel for locations provided to city Taltal
Hotel found !!


Hotel not found ... skipping.
Getting hotel for locations provided to city Malatya
Hotel not found ... skipping.
Getting hotel for locations provided to city Santa Cruz De Tenerife
Hotel found !!
Getting hotel for locations provided to city Serzedo
Hotel found !!
Getting hotel for locations provided to city Gonbad-E Qabus
Hotel found !!
Getting hotel for locations provided to city Humaita
Hotel found !!
Getting hotel for locations provided to city Afzalpur
Hotel found !!
Getting hotel for locations provided to city Poum
Hotel found !!
Getting hotel for locations provided to city Amapa
Hotel not found ... skipping.
Getting hotel for locations provided to city Werda
Hotel not found ... skipping.
Getting hotel for locations provided to city Bredy
Hotel found !!
Getting hotel for locations provided to city Olafsvik
Hotel found !!
Getting hotel for locations provided to city Keningau
Hotel found !!
Getting hotel for locations provided to city Monrovia
Hotel found !!
Getting hotel for locati

In [33]:
# 7. Drop the rows where there is no Hotel Name.
#replace empty for null
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("",np.nan)

# if there is any null value then drop the line
if (hotel_df["Hotel Name"].count() < hotel_df["City"].count()) :
    hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,74.68,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
4,Banjar,ID,78.66,scattered clouds,-8.1900,114.9675,The Hamsa Resort
6,Ferrol,ES,62.67,scattered clouds,43.4832,-8.2369,Parador de Ferrol
7,Praya,CV,77.54,few clouds,14.9215,-23.5087,LT Aparthotel
8,Saint-Philippe,RE,65.75,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
676,Okitipupa,NG,73.13,overcast clouds,6.5047,4.7839,Coastal Hotel & Suites
677,Atambua,ID,73.58,scattered clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
678,Ahipara,NZ,58.39,clear sky,-35.1667,173.1667,Ahipara Holiday Park
679,Byron Bay,AU,69.06,clear sky,-28.6500,153.6167,Backpackers Inn on the Beach


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name </dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather<dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))